## Automate Reading BOM Json and updating pom.xml


### Future Upgrades

- Call the Tidelift API instead of reading the bom.json
- After pom.xml is updated, automate Pull request
- Explore tools like snyk that does it automatically.

### Function to update pom.xml 


In [58]:

import xml.etree.ElementTree as ET

def update_dependency_version(pom_file, dependency_artifact_id, new_version):
    """
    Updates the version of a dependency with the specified artifact ID in the
    specified pom.xml file to the specified new version.
    """

    # Parse the pom.xml file
    tree = ET.parse(pom_file)
    root = tree.getroot()

    # Find the dependency you want to update
    dependency = root.find(f".//dependency/[artifactId='{dependency_artifact_id}']")
    # print (dependency, new_version)

    if dependency is None or new_version is '':
        print (f"Dependency with artifactId '{dependency_artifact_id} skipped")
        #raise ValueError(f"Dependency with artifactId '{dependency_artifact_id}' not found.")
    else:
        # Update the version of the dependency
        dependency.find("version").text = new_version

        # Write the updated pom.xml file
        tree.write(pom_file)

        print(f"Dependency with artifactId '{dependency_artifact_id}' updated to version '{new_version}' in '{pom_file}'.")

### Process the Vulnerability Report
- Load json 
- Covert to dataframe
- Filter datframe for hackathon project
- iterate and call update pom function 


In [64]:
import pandas as pd
import json

# Set the path to the JSON file
json_file = "bom.json"

# Load the JSON file into a Python object
with open(json_file, "r") as f:
    data = json.load(f)

# Extract the node you want to read into a DataFrame
node = data["report"]

# Convert the node to a Pandas DataFrame
df = pd.json_normalize(node)

#filter only interested project
required_rows_df = df[["package_name", "affected_project","version_guidance"]]
only_project_df = required_rows_df.loc[required_rows_df["affected_project"] == "G0H-INNERSOURCE-HACKATHON"].drop_duplicates()


# Print the DataFrame
print(only_project_df)

                                    package_name           affected_project  \
0    com.fasterxml.jackson.core:jackson-databind  G0H-INNERSOURCE-HACKATHON   
6                     mysql:mysql-connector-java  G0H-INNERSOURCE-HACKATHON   
14                                   junit:junit  G0H-INNERSOURCE-HACKATHON   
16                  org.hibernate:hibernate-core  G0H-INNERSOURCE-HACKATHON   
37                                   dom4j:dom4j  G0H-INNERSOURCE-HACKATHON   
74                    org.springframework:spring  G0H-INNERSOURCE-HACKATHON   
126      commons-collections:commons-collections  G0H-INNERSOURCE-HACKATHON   

    version_guidance  
0             2.15.0  
6             8.0.33  
14            4.13.2  
16       6.2.2.Final  
37                    
74            5.3.27  
126            3.2.2  


In [65]:
## Loop thru the vulnerabilities dataframe and update pom

pom_file = "pom.xml"


for index, row in only_project_df.iterrows():
    if row.package_name is not None and ":" in row.package_name:
        dependency_artifact_id = row.package_name.split(':')[1]
    else:
        dependency_artifact_id = row.package_name
    new_version = row.version_guidance
    skip_list = ['spring']
    if dependency_artifact_id not in skip_list:
        update_dependency_version(pom_file, dependency_artifact_id, new_version)
    # validate_pom()

Dependency with artifactId 'jackson-databind' updated to version '2.15.0' in 'pom.xml'.
Dependency with artifactId 'mysql-connector-java' updated to version '8.0.33' in 'pom.xml'.
Dependency with artifactId 'junit' updated to version '4.13.2' in 'pom.xml'.
Dependency with artifactId 'hibernate-core skipped
Dependency with artifactId 'dom4j skipped
Dependency with artifactId 'commons-collections' updated to version '3.2.2' in 'pom.xml'.


In [66]:
## Read pom.xml and print dependency
import xml.etree.ElementTree as ET

# Set the path to the pom.xml file
pom_file = "pom.xml"

# Parse the pom.xml file
tree = ET.parse(pom_file)
root = tree.getroot()

# Find all dependencies in the pom.xml file
dependencies = root.findall(".//dependency")

# Print the dependencies and versions
for dependency in dependencies:
    group_id = dependency.find("groupId").text
    artifact_id = dependency.find("artifactId").text
    version = dependency.find("version").text
    print(f"{group_id}:{artifact_id}:{version}")

junit:junit:4.13.2
org.springframework:spring:2.5.6
cglib:cglib:2.2
mysql:mysql-connector-java:8.0.33
org.hibernate:hibernate-annotations:3.4.0.GA
org.hibernate:hibernate-commons-annotations:3.2.0.Final
dom4j:dom4j:1.6.1
commons-logging:commons-logging:1.1.1
commons-collections:commons-collections:3.2.2
antlr:antlr:2.7.7
com.fasterxml.jackson.core:jackson-databind:2.15.0
com.fasterxml.jackson.core:jackson-annotations:2.0.0
com.fasterxml.jackson.core:jackson-core:2.0.0
org.springframework:spring-expression:5.1.0.RELEASE
org.springframework:spring-web:5.1.0.RELEASE
org.springframework:spring-beans:5.1.0.RELEASE
org.springframework:spring-core:5.1.0.RELEASE
